In [108]:
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, pipeline
import seaborn as sns
import pickle

In [97]:
path = r"https://raw.githubusercontent.com/abulbasar/data/master/kaggle-houseprice/data_combined_cleaned.csv"

df = pd.read_csv(path)
df = df[~np.isnan(df.SalesPrice)]
df = pd.get_dummies(df)
temp = df

In [41]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X , y , test_size = 0.3 ,random_state = 1)

In [70]:
pipeLr = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree = 2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("lr", linear_model.LinearRegression())
])
pipe.fit(X_train, y_train)
lr = pipe.score(X_test, y_test)
print("LinearModel -> ", lr)

LinearModel ->  0.874410244957


In [100]:
pipeLasso = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree = 2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("fit", linear_model.Lasso(alpha=0.1))
])
pipeLasso.fit(X_train, y_train)
lasso = pipeLasso.score(X_test, y_test)
print("Lasso -> ", lasso)

Lasso ->  0.777199347745


In [101]:
pipeElastic = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree = 2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("fit", linear_model.ElasticNet(alpha=1, l1_ratio=0.45))
])
pipeElastic.fit(X_train, y_train)
pipeElastic.score(X_test, y_test)
El_score = pipeElastic.score(X_test, y_test)
print("Elasticnet -> " ,El_score)

C:\Users\sourabh.v\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Elasticnet ->  0.879255059573


In [68]:
pipeRidge = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree = 2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("lr", linear_model.Ridge(alpha=0.1))
])
pipeRidge.fit(X_train, y_train)
Rid_score = pipeRidge.score(X_test, y_test)
print("Ridge -> " ,Rid_score)

Ridge ->  0.874410244957


In [84]:

y = df.SalesPrice
del df["SalesPrice"]
del df["Id"]
X= pd.get_dummies(df)
X.head()
scores = model_selection.cross_val_score(cv=5, estimator=pipeLr ,X=X ,y=y)

In [72]:
print(scores)

[ 0.82447937  0.74938812  0.8086129   0.84140093  0.48623797]


In [102]:
param_grid = {
    "fit__alpha" : np.arange(.25,.50),
    "fit__l1_ratio" : np.arange(0.30 , 0.45)
}

In [103]:

ga = model_selection.GridSearchCV(cv=5, estimator=pipeElastic, param_grid=param_grid)
ga.fit(X ,y)

C:\Users\sourabh.v\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\sourabh.v\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\sourabh.v\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\sourabh.v\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('fit', ElasticNet(alpha=1, copy_X=True, fit_intercept=True, l1_ratio=0.45,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'fit__alpha': array([ 0.25]), 'fit__l1_ratio': array([ 0.3])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [104]:
ga.best_estimator_

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('fit', ElasticNet(alpha=0.25, copy_X=True, fit_intercept=True,
      l1_ratio=0.29999999999999999, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False))])

In [105]:
ga.best_params_

{'fit__alpha': 0.25, 'fit__l1_ratio': 0.29999999999999999}

In [119]:
x = open("myModel" , "wb")
pickle.dump(ga,x)

In [121]:
x = open("myModel" , "rb")
ga = pickle.load(x)

In [122]:
ga.best_score_

0.73743501070039374